In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the CSV file with prime factors and ranks
file_path = '/Users/mac/Desktop/mahdi_313.csv'
df = pd.read_csv(file_path)

# Function to assign weights based on prime numbers
def assign_weights(rankings, prime_weights, default_weight=1.0):
    return np.array([prime_weights.get(rank, default_weight) for rank in rankings])

# Define the optimizer function based on the ranking of abjad
def prime_weighted_objective_function(weights, features, rankings, prime_weights, default_weight):
    adjusted_features = features * weights
    assigned_weights = assign_weights(rankings, prime_weights, default_weight)
    scores = np.sum(adjusted_features * assigned_weights[:, None], axis=1)
    return -np.mean(scores)  # We minimize negative score to maximize positive score

# Function to optimize prime weights and feature weights
def optimize_prime_weights_and_features(X_train, y_train, rankings, initial_prime_weights, initial_feature_weights, default_weight=1.0):
    def objective(combined_weights):
        feature_weights = combined_weights[:X_train.shape[1]]
        prime_weights = combined_weights[X_train.shape[1]:]
        prime_weights_dict = {rank: weight for rank, weight in zip(np.unique(rankings), prime_weights)}
        return prime_weighted_objective_function(feature_weights, X_train, rankings, prime_weights_dict, default_weight)
    
    # Initial combined weights
    initial_combined_weights = np.concatenate([initial_feature_weights, initial_prime_weights])

    # Bounds for feature weights and prime weights
    bounds = [(0.01, 10)] * len(initial_combined_weights)

    result = minimize(objective, initial_combined_weights, method='L-BFGS-B', bounds=bounds)
    optimal_combined_weights = result.x

    optimal_feature_weights = optimal_combined_weights[:X_train.shape[1]]
    optimal_prime_weights = optimal_combined_weights[X_train.shape[1]:]
    
    prime_weights_dict = {rank: weight for rank, weight in zip(np.unique(rankings), optimal_prime_weights)}
    return optimal_feature_weights, prime_weights_dict

# Function to run the optimization and calculate MSE for a given stock ticker
def run_simulation(stock_ticker):
    # Fetch historical data
    data = yf.download(stock_ticker, start='2023-06-10', end='2024-06-10')

    if data.empty:
        print(f"No data found for {stock_ticker}")
        return

    # Feature selection
    features = data[['Open', 'High', 'Low', 'Close', 'Volume']].values
    target = data['Close'].values

    # Add the ranking feature from the CSV
    rankings = df['Rank'].values[:len(features)]

    # Normalize features
    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Initial weights for optimization
    initial_feature_weights = np.ones(X_train.shape[1])
    initial_prime_weights = np.ones(len(np.unique(rankings)))  # Initial weights for each unique prime rank

    # Optimize prime weights and feature weights
    optimal_feature_weights, optimal_prime_weights = optimize_prime_weights_and_features(
        X_train, y_train, rankings[:len(X_train)], initial_prime_weights, initial_feature_weights
    )

    # Adjust features based on the optimized weights
    X_train_opt = X_train * optimal_feature_weights
    X_test_opt = X_test * optimal_feature_weights

    # Train the Optimized Linear Regression model
    model_lr = LinearRegression()
    model_lr.fit(X_train_opt, y_train)
    y_pred_lr = model_lr.predict(X_test_opt)
    mse_lr = mean_squared_error(y_test, y_pred_lr)

    # Print the MSE of the new model
    print(f'{stock_ticker} Optimized Linear Regression MSE: {mse_lr}')
    print(f'Optimal Prime Weights: {optimal_prime_weights}')
    print(f'Optimal Feature Weights: {optimal_feature_weights}')

# List of selected stocks
selected_stocks = ['SPY', 'META', 'GOOGL', 'MSFT', 'NVDA', 'AAPL']

# Run the simulation for selected stocks
for stock in selected_stocks:
    run_simulation(stock)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

SPY Optimized Linear Regression MSE: 0.0
Optimal Prime Weights: {8: 0.01, 11: 0.01, 12: 10.0, 14: 0.01, 15: 0.01, 16: 0.01, 22: 0.01, 23: 10.0, 26: 10.0, 27: 0.01, 28: 10.0, 29: 10.0, 30: 0.01, 31: 0.01, 32: 0.01, 33: 10.0, 34: 10.0, 35: 10.0, 36: 10.0, 37: 10.0, 38: 0.01, 39: 10.0, 40: 0.01, 41: 0.01, 42: 0.01, 43: 10.0, 44: 0.01, 45: 0.01, 46: 0.01}
Optimal Feature Weights: [10. 10. 10. 10. 10.]



[*********************100%%**********************]  1 of 1 completed

META Optimized Linear Regression MSE: 3.877409121342317e-28
Optimal Prime Weights: {8: 0.01, 11: 0.01, 12: 10.0, 14: 0.01, 15: 0.01, 16: 0.01, 22: 0.01, 23: 10.0, 26: 10.0, 27: 0.01, 28: 10.0, 29: 10.0, 30: 0.01, 31: 0.01, 32: 0.01, 33: 10.0, 34: 10.0, 35: 10.0, 36: 10.0, 37: 10.0, 38: 0.01, 39: 10.0, 40: 0.01, 41: 0.01, 42: 0.01, 43: 10.0, 44: 0.01, 45: 0.01, 46: 0.01}
Optimal Feature Weights: [10.   10.   10.   10.    0.01]



[*********************100%%**********************]  1 of 1 completed

...


[*********************100%%**********************]  1 of 1 completed

MSFT Optimized Linear Regression MSE: 1.2278462217584005e-27
Optimal Prime Weights: {8: 0.01, 11: 0.01, 12: 10.0, 14: 0.01, 15: 0.01, 16: 0.01, 22: 0.01, 23: 10.0, 26: 10.0, 27: 0.01, 28: 0.01, 29: 10.0, 30: 0.01, 31: 0.01, 32: 0.01, 33: 10.0, 34: 10.0, 35: 10.0, 36: 0.01, 37: 10.0, 38: 0.01, 39: 10.0, 40: 10.0, 41: 0.01, 42: 0.01, 43: 10.0, 44: 0.01, 45: 10.0, 46: 0.01}
Optimal Feature Weights: [10. 10. 10. 10. 10.]



[*********************100%%**********************]  1 of 1 completed

NVDA Optimized Linear Regression MSE: 8.683780844672898e-29
Optimal Prime Weights: {8: 0.01, 11: 0.01, 12: 10.0, 14: 0.01, 15: 0.01, 16: 0.01, 22: 0.01, 23: 10.0, 26: 10.0, 27: 0.01, 28: 10.0, 29: 10.0, 30: 0.01, 31: 0.01, 32: 10.0, 33: 10.0, 34: 10.0, 35: 10.0, 36: 10.0, 37: 0.01, 38: 0.01, 39: 10.0, 40: 10.0, 41: 10.0, 42: 0.01, 43: 10.0, 44: 0.01, 45: 10.0, 46: 0.01}
Optimal Feature Weights: [10. 10. 10. 10. 10.]


AAPL Optimized Linear Regression MSE: 0.0
Optimal Prime Weights: {8: 10.0, 11: 0.01, 12: 0.01, 14: 10.0, 15: 0.01, 16: 0.01, 22: 0.01, 23: 10.0, 26: 0.01, 27: 10.0, 28: 10.0, 29: 0.01, 30: 0.01, 31: 10.0, 32: 0.01, 33: 10.0, 34: 0.01, 35: 10.0, 36: 0.01, 37: 10.0, 38: 0.01, 39: 10.0, 40: 10.0, 41: 0.01, 42: 10.0, 43: 0.01, 44: 10.0, 45: 0.01, 46: 10.0}
Optimal Feature Weights: [10.   10.   10.   10.    0.01]
